# Tutorial 5: Memory- and time-efficient solving of ME-models

In this tutorial we will convert the ME-model object to an NLP mathematical representation to save memory and time in simulating many conditions.

## Import libraries

In [1]:
import coralme
from helpers import get_nlp,optimize

## Load

Load the ME-model coming out of the Troubleshooter

In [2]:
me = coralme.io.json.load_json_me_model("../Tutorial 1 - Full reconstruction/MEModel-step3-bsubtilis-TS.json")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-03


Adding Metabolites into the ME-model...                                    : 100.0%|██████████|  4436/ 4436 [00:00<00:00]
Adding ProcessData into the ME-model...                                    : 100.0%|██████████|  4488/ 4488 [00:00<00:00]
Adding Reactions into the ME-model...                                      : 100.0%|██████████|  7466/ 7466 [00:12<00:00]
Updating ME-model Reactions...                                             : 100.0%|██████████|  6121/ 6121 [00:17<00:00]


## Convert to NLP problem

The ME-model object *me* is a big object containing all data and metadata. This is not necessary when performing large-scale simulations, such as gene knockouts, or growth simulations under hundreds of conditions.

So, in these cases we only need the mathematical problem representing the ME-model, which is *nlp*.

In [3]:
nlp = get_nlp(me)

## Retrieve metabolite and reaction indexes

The *nlp* now contains the mathematical representation, very similar to a struct object of the COBRA Toolbox in MATLAB. Similarly, reactions and metabolites are now accessed from integer indexes. We can create a dictionary from the original model to map reaction ids to indexes

In [4]:
rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}

From now on, *me* is no longer necessary and can be deleted to save memory usage. This is especially helpful when running parallelized simulations.

In [5]:
# del me

## Solving the MEModel vs. NLP

Now we can call the modified *optimize* function in *helpers*. This function was modified from the me.optimize() function of a coralme.core.model.MEModel.

Here you can see the speed-up when solving from scratch and solving from the NLP. The speed-up is even more noticeable with bigger models, as lamdifying a longer list of constraints will take much longer.

### ME-model

In [6]:
%%time
me.optimize(max_mu = 0.1, min_mu = 0., maxIter = 100, lambdify = True,
		tolerance = 1e-6, precision = 'quad', verbose = True)

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0500000000000000	Optimal
        2	0.0750000000000000	Optimal
        3	0.0875000000000000	Optimal
        4	0.0937500000000000	Not feasible
        5	0.0906250000000000	Optimal
        6	0.0921875000000000	Not feasible
        7	0.0914062500000000	Optimal
        8	0.0917968750000000	Optimal
        9	0.0919921875000000	Not feasible
       10	0.0918945312500000	Optimal
       11	0.0919433593750000	Not feasible
       12	0.0919189453125000	Not feasible
       13	0.0919067382812500	Optimal
       14	0.0919128417968750	Not feasible
       15	0.0919097900390625	Optimal
       16	0.0919113159179688	Not feasible
 

True

### NLP

In [7]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 100,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0500000000000000	Optimal
        2	0.0750000000000000	Optimal
        3	0.0875000000000000	Optimal
        4	0.0937500000000000	Not feasible
        5	0.0906250000000000	Optimal
        6	0.0921875000000000	Not feasible
        7	0.0914062500000000	Optimal
        8	0.0917968750000000	Optimal
        9	0.0919921875000000	Not feasible
       10	0.0918945312500000	Optimal
       11	0.0919433593750000	Not feasible
       12	0.0919189453125000	Not feasible
       13	0.0919067382812500	Optimal
       14	0.0919128417968750	Not feasible
       15	0.0919097900390625	Optimal
       16	0.0919113159179688	Not feasible
       17	0.0919105529785156	Not feasible
CPU times: user 1min 18s, sys: 49.1 ms, total: 1min 18s
Wall time: 1min 18s


**Speed-up of complete calculation from ~86 seconds to ~78 seconds!**

## Re-using the basis for even more speed-up

We can re-use a basis from a previously successful simulation to warm-start the first iteration and save even more time! 

### Re-using basis

In [9]:
%%time
sol,_ = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 1,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0500000000000000	Optimal
CPU times: user 6.66 s, sys: 3.95 ms, total: 6.66 s
Wall time: 6.66 s


### Cold start

In [10]:
%%time
sol,_ = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 1, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0500000000000000	Optimal
CPU times: user 56.7 s, sys: 4.7 ms, total: 56.7 s
Wall time: 56.6 s


**Speed-up of first iteration from ~56 seconds to ~7 seconds!**

### Full calculation

In [11]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 100, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0500000000000000	Optimal
        2	0.0750000000000000	Optimal
        3	0.0875000000000000	Optimal
        4	0.0937500000000000	Not feasible
        5	0.0906250000000000	Optimal
        6	0.0921875000000000	Not feasible
        7	0.0914062500000000	Optimal
        8	0.0917968750000000	Optimal
        9	0.0919921875000000	Not feasible
       10	0.0918945312500000	Optimal
       11	0.0919433593750000	Not feasible
       12	0.0919189453125000	Not feasible
       13	0.0919067382812500	Optimal
       14	0.0919128417968750	Not feasible
       15	0.0919097900390625	Optimal
       16	0.0919113159179688	Not feasible
       17	0.0919105529785156	Not feasible
CPU times: user 34.5 s, sys: 20 ms, total: 34.6 s
Wall time: 34.5 s


**Speed-up of complete calculation from ~84 seconds to ~34 seconds!**

## Modifying the NLP

As previously mentioned, the NLP resembles a struct object of the COBRA Toolbox. The model is stored as a collection of vectors and matrices representing stoichiometries, bounds and other variables needed by the solvers.

The relevant properties are:
* **xu**: Upper bounds
* **xl**: Lower bounds
* **S**: Stoichiometric matrix (Metabolites x Reactions)

The carbon source right now is Glucose, so we will change its bound to -10 to try to achieve maximum growth rate. 

**Note that bounds contain *lambdify* objects, not floats!**

In [12]:
nlp.xl[rxn_index_dct["EX_glc__D_e"]] = lambda x:-10

In [13]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.5, min_mu = 0., maxIter = 100, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.2500000000000000	Optimal
        2	0.3750000000000000	Optimal
        3	0.4375000000000000	Not feasible
        4	0.4062500000000000	Optimal
        5	0.4218750000000000	Optimal
        6	0.4296875000000000	Not feasible
        7	0.4257812500000000	Optimal
        8	0.4277343750000000	Optimal
        9	0.4287109375000000	Optimal
       10	0.4291992187500000	Not feasible
       11	0.4289550781250000	Optimal
       12	0.4290771484375000	Optimal
       13	0.4291381835937500	Not feasible
       14	0.4291076660156250	Optimal
       15	0.4291229248046875	Not feasible
       16	0.4291152954101562	Not feasible
       17	0.4291114807128906	Not feasible
       18	0.4291095733642578	Optimal
       19	0.4291105270385742	Optimal
CPU times: user 40.4 s, sys: 11.7 ms, total: 40.4 s
Wall time: 40.4 s


## Modifying the NLP from a dictionary of new bounds

Make sure to follow this method so that the lambda does not store pointers to a variable but rather a fixed constant (if that is what you want).

In [14]:
def set_exchanges(nlp,dct):
    for k,v in dct.items():
        nlp.xl[rxn_index_dct[k]] = lambda _,x=v:x

In [15]:
exchanges = {
    "EX_glc__D_e" : -10,
    "EX_o2_e" : -0.6,
    "EX_fru_e" : -5,
}

In [16]:
set_exchanges(nlp,exchanges)

## Inspecting the solution

The function returns a cobra.Solution object just like the one stored in me.solution. For more details inspecting *sol*, refer to Tutorial 3.

In [17]:
sol

<Solution 0.429 at 0x7f8a37eada80>